In [1]:
import pandas as pd
import numpy as np
import keras
import tensorflow
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense


data = pd.read_csv("train.txt", sep=';')
data.columns = ["Text", "Emotions"]
print(data.head())

FileNotFoundError: [Errno 2] No such file or directory: 'train.txt'

In [ ]:
texts = data["Text"].tolist()
labels = data["Emotions"].tolist()

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

In [ ]:
sequences = tokenizer.texts_to_sequences(texts)
max_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_length)

In [ ]:
sequences = tokenizer.texts_to_sequences(texts)

In [ ]:
padded_sequences

array([[   0,    0,    0, ...,    3,   21, 1254],
       [   0,    0,    0, ...,    2,  494,  437],
       [   0,    0,    0, ...,   29,    5, 3496],
       ...,
       [   0,    0,    0, ...,    3,  101, 1331],
       [   0,    0,    0, ...,  339,    8,   42],
       [   0,    0,    0, ...,   25, 3585,   12]])

In [ ]:
# Encode the string labels to integers
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

In [ ]:
# One-hot encode the labels
one_hot_labels = keras.utils.to_categorical(labels)

In [ ]:
one_hot_labels

array([[0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       ...,
       [0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.]], dtype=float32)

In [ ]:
# Split the data into training and testing sets
xtrain, xtest, ytrain, ytest = train_test_split(padded_sequences,
                                                one_hot_labels,
                                                test_size=0.2)

In [ ]:
one_hot_labels[0]

array([0., 0., 0., 0., 1., 0.], dtype=float32)

In [ ]:
# Define the model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1,
                    output_dim=128, input_length=max_length))
model.add(Flatten())
model.add(Dense(units=128, activation="relu"))
model.add(Dense(units=len(one_hot_labels[0]), activation="softmax"))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(xtrain, ytrain, epochs=10, batch_size=32, validation_data=(xtest, ytest))

Epoch 1/10
400/400 [==============================] - 25s 62ms/step - loss: 1.3679 - accuracy: 0.4642 - val_loss: 0.8908 - val_accuracy: 0.6825
Epoch 2/10
400/400 [==============================] - 25s 63ms/step - loss: 0.3552 - accuracy: 0.8867 - val_loss: 0.5729 - val_accuracy: 0.8134
Epoch 3/10
400/400 [==============================] - 23s 58ms/step - loss: 0.0597 - accuracy: 0.9845 - val_loss: 0.5650 - val_accuracy: 0.8325
Epoch 4/10
400/400 [==============================] - 24s 60ms/step - loss: 0.0223 - accuracy: 0.9955 - val_loss: 0.5972 - val_accuracy: 0.8328
Epoch 5/10
400/400 [==============================] - 21s 52ms/step - loss: 0.0178 - accuracy: 0.9966 - val_loss: 0.6095 - val_accuracy: 0.8359
Epoch 6/10
400/400 [==============================] - 21s 53ms/step - loss: 0.0135 - accuracy: 0.9975 - val_loss: 0.6500 - val_accuracy: 0.8319
Epoch 7/10
400/400 [==============================] - 22s 56ms/step - loss: 0.0125 - accuracy: 0.9978 - val_loss: 0.7074 - val_accuracy:

In [ ]:
input_text = "i am ever feeling nostalgic about the fireplace i will know that it is still on the property"

# Preprocess the input text
input_sequence = tokenizer.texts_to_sequences([input_text])
padded_input_sequence = pad_sequences(input_sequence, maxlen=max_length)
prediction = model.predict(padded_input_sequence)
predicted_label = label_encoder.inverse_transform([np.argmax(prediction[0])])
print(predicted_label[0])

love
